# 🦐 对半网格搜索HalvingSearchCV

决定网格搜索运算速度的因子：
1. 参数空间大小
2. 数据量大小

sklearn呈现了两种优化方式：调整搜索空间（随机网格搜索）和调整每次训练数据（对半网格搜索）

依据：任意子集的分布都与全集D的分布类似时，参数在子集和全集上的表现一致。

因此，需要权衡子集大小和计算效率的问题。对半网格搜索的流程：

1. 从全集中无放回随机抽样一个很小的子集d0，并验证全部组合的性能，根据d0上的验证结果淘汰评分排在后1/2的参数组合
2. 从全集中无放回随机抽样一个比d0大一倍的子集d1，并在d1上验证剩余一半的参数组合，再淘汰后1/2
3. 再从全集中无放回抽样一个比d1大一倍的子集d2，在d2上验证剩下1/4的参数组合，再将后1/2淘汰
4. 持续循环

| 迭代次数 | 子集样本量 | 参数组合数              |
| -------- | ---------- | ----------------------- |
| 1        | S          | C                       |
| 2        | 2S         | 1/2C                    |
| 3        | 4S         | 1/4C                    |
| 4        | 8S         | 1/8C                    |
| ...      |            | 当C无法除尽时，向上取整 |

当参数组合只剩下一组、或剩余可用数据不足，循环停止。

局限性：

1. 子集越大，和全集越相似，但在开始时淘汰了一半的参数。
2. 不放回的方式，需要的样本量必须很大

函数：

```python
from sklearn.model_selection import HalvingGridSearchCV
```


| Name                     | Description                                              |
| ------------------------ |----------------------------------------------------------|
| `estimator`              | 评估器、调参对象                                                 |
| `param_grid`             | 参数空间，`dict`、`list` of `dict`                             |
| `factor`                 | 每轮迭代中新增样本量的比例，也是每轮迭代后剩下的参数组合比例。如`factor=2`，下一轮是上一轮样本量的两倍 |
| `resource`               | 设置每轮迭代中增加的验证资源的类型                                        |
| `max_resources`          | 一次迭代中，允许被用来验证任意参数组合的最大样本量                                |
| `min_resources`          | 首次迭代时，用于验证参数组合的样本量                                       |
| `aggressive_elimination` | 是否以全部数据被使用完作为停止搜索的指标，如果不是，则采取措施                          |
| `cv`                     | 交叉验证折数                                                   |
| `scoring`                | 评价指标，支持多个输出                                              |
| `refit`                  | 挑选评估指标和最佳参数，在完整数据集上训练                                    |
| `error_score`            | 网格搜索报错时返回结果，选择`raise`时直接报错并中断训练过程，其他情况会显示警告信息后继续完成训练     |
| `return_train_score`     | 交叉验证是否显示训练集中参数得分                                         |
| `random_state`           | 随机数种子                                                    |
| `n_jobs`                 | 线程数                                                      |
| `verbose`                | 输出工作日志                                                   |